## Demonstrate the use of Precision Timing Capability
In this experiment, we will start a tcpdump process to capture all ICMP request packets with a nano-second high precision timestamp obtained from the NIC/PHC Clock into `PCAP` files. We will then send 5 ping packets from Node1 to Node4. Once the ping operation is complete, we will gather the pcap files and perform an analysis 
on the packet's timestamps by tracking each packet as it flows though all the hops to reach its destination.

## Install python-scapy library

In [ ]:
%%bash
pip install scapy
pip install --upgrade plotly
chmod +x tools/*

## Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
import requests
from scapy.all import *

try:
    fablib = fablib_manager()
                     
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

## Get Slice Name

In [ ]:
slice_name=f"Slice for KNIT7 Precision Timing Tutorial"
slice = fablib.get_slice(name = slice_name)
slice.show()
slice.list_nodes()
slice.list_networks()
slice.list_interfaces()

## Experiment Setup
### Perform the trials - (a) Time skewed on a node (b) Time synchronized using PTP on all nodes

1. Upload  bash scripts in tools directory that will perform tcpdumps
2. Start tcpdump on all nodes
3. Start a series of 5 pings from Node1 to Node4
4. Stop the tcpdump operation.
5. Gather all pcap files from the tcpdump operation

### Trial 1 (Time skewed on Node3)
#### Disable PTP Synchronization and Skew Clock time on NICs Node3

In [ ]:
import datetime
node3 = slice.get_node(name='node3')
node3_interfaces = node3.get_interfaces()
for interface_obj in node3_interfaces:
    interface = interface_obj.get_device_name()
    print (f"Working on node3->{interface}")
    print (f" Stopping PTP Synchronization on node3->{interface}")
    stdout,stderr = node3.execute(f'sudo systemctl stop phc2sys@{interface}.service')
    
    print (f" Get Time from node3->{interface} CLOCK/PHC")
    stdout,stderr = node3.execute("sudo ethtool -T "+interface+"|grep 'PTP Hardware Clock:'|awk '{print $4}'",quiet=True)
    ptp_index = stdout.strip()
    stdout,stderr = node3.execute(f"sudo phc_ctl /dev/ptp{ptp_index} get")
    curr_time = float(stdout.split()[4])
    print (f" Skew Time on node3->{interface} CLOCK/PHC ...")
    stdout,stderr = node3.execute("sudo phc_ctl /dev/ptp"+ptp_index+" set " + str(int(curr_time)) + ";sudo phc_ctl /dev/ptp"+ptp_index+" cmp")

print (f"Time Skew Operation Completed\n\n")

#### Start traffic generator and caputure packets

In [ ]:
nodes = slice.get_nodes()
mode = 'skewed'
node1 = None
for node in nodes:
    node.upload_directory('tools','~/')
    iface_name = None
    node_name = node.get_name()
    if(node_name in ['node1']):
        iface_name = (node.get_interfaces()[0]).get_physical_os_interface_name()
        node1 = node
    elif(node_name in ['node4']):
        iface_name = (node.get_interfaces()[0]).get_physical_os_interface_name()
    elif(node_name in ['node2']):
        iface_name = (node.get_interface(network_name='net1')).get_physical_os_interface_name()
    else:
        iface_name = (node.get_interface(network_name='net2')).get_physical_os_interface_name()
    print (f"Starting tcpdump on {node_name}")
    node.execute_thread(f"sudo -b tools/start_dump.sh {node_name} {iface_name} {mode}")
    
stdout,stderr = node1.execute(f"ping -c10 node4")

for node in nodes:
    node_name = node.get_name()
    pcap_filename = f'{node_name}_{mode}.pcap'
    print (f"Stop TCPDUMP and Download pcap file to {node_name}_{mode}.pcap")
    stdout,stderr = node.execute(f"sudo -b tools/stop_dump.sh")
    node.download_file(pcap_filename,pcap_filename)

### Trial 2 (Time synchronized using PTP)

#### Start PTP synchronization again on Node3

In [ ]:
node3 = slice.get_node(name='node3')
node3_interfaces = node3.get_interfaces()
for interface_obj in node3_interfaces:
    interface = interface_obj.get_device_name()
    print (f"Working on node3->{interface}")
    print (f" Starting PTP Synchronization on node3->{interface}")
    stdout,stderr = node3.execute(f'sudo systemctl start phc2sys@{interface}.service;sleep 5')
    print (f" Get Time from node3->{interface} CLOCK/PHC")
    stdout,stderr = node3.execute("sudo ethtool -T "+interface+"|grep 'PTP Hardware Clock:'|awk '{print $4}'",quiet=True)
    ptp_index = stdout.strip()
    stdout,stderr = node3.execute(f"sudo phc_ctl /dev/ptp{ptp_index} get;sudo phc_ctl /dev/ptp{ptp_index} cmp")

print (f"Time Sync Operation Completed\n\n")

#### Start traffic generator and caputure packets

In [ ]:
nodes = slice.get_nodes()
mode = 'synced'
node1 = None
for node in nodes:
    iface_name = None
    node_name = node.get_name()
    if(node_name in ['node1']):
        iface_name = (node.get_interfaces()[0]).get_physical_os_interface_name()
        node1 = node
    elif(node_name in ['node4']):
        iface_name = (node.get_interfaces()[0]).get_physical_os_interface_name()
    elif(node_name in ['node2']):
        iface_name = (node.get_interface(network_name='net1')).get_physical_os_interface_name()
    else:
        iface_name = (node.get_interface(network_name='net2')).get_physical_os_interface_name()
    print (f"Starting tcpdump on {node_name}")
    node.execute_thread(f"sudo -b tools/start_dump.sh {node_name} {iface_name} {mode}")
    
stdout,stderr = node1.execute(f"ping -c10 node4")

for node in nodes:
    node_name = node.get_name()
    pcap_filename = f'{node_name}_{mode}.pcap'
    print (f"Stop TCPDUMP and Download pcap file to {node_name}_{mode}.pcap")
    stdout,stderr = node.execute(f"sudo -b tools/stop_dump.sh")
    node.download_file(pcap_filename,pcap_filename)

## Analyze packet captures
As the ping packets traverse thru node2 and node3 to reach node4, we collect pcap files at each hop on the incomming interfaces. Using the packet traces we identify each packet and compare the packet capture timestamps. If the clocks on all NICS were synchronized via GPS clocks, then we should see timestamps incrementing. If for some reason the clock on any one of the NICs is not synchronized, the timestamps may not sequentially match up and lead to miscalcuations in computing transit times at intermediate(or end) hops.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio


pio.renderers.default = 'iframe'  # Necessary for displaying graphs. Will be cleaned at the end.
pd.set_option('display.precision', 9)

In [ ]:
nodes = slice.get_nodes()
node_names = [node.get_name() for node in nodes]

In [ ]:
# Since displaying graphs within a loop does not work, we have to "save" the parsed data

parsed_data={}

for mode in ['skewed','synced']:
    parsed_packets = {}
    print (f'************** ANALYSIS for TRIAL : Node is time {mode} ***************************')
    for node in nodes:
        node_name = node.get_name()
        packets = PcapReader(f'{node_name}_{mode}.pcap')
        for packet in packets:
            new_list = []
            if (str(packet[IP].id) not in parsed_packets.keys()):
                parsed_packets.update({ str(packet[IP].id): {node_name : {'src': packet[IP].src,'dst': packet[IP].dst,'timestamp': packet.time }}})
            else:
                parsed_packets[str(packet[IP].id)].update({node_name : {'src': packet[IP].src,'dst': packet[IP].dst,'timestamp': packet.time }})
    
    parsed_data[mode]=parsed_packets
    
    packets = PcapReader(f'node1_{mode}.pcap')
    for packet in packets:
        if len(parsed_packets[str(packet[IP].id)]) == len(nodes):
            if (packet[IP].src == packet[IP].src) and (packet[IP].dst == packet[IP].dst):
                print (f"{packet.summary()} with ICMP SEQ# {packet[ICMP].seq} \n\
        found exiting node1 at {packet.time}")
                for other_node in ['node2','node3','node4']:
                    print(f"\
    and then entering {other_node} at {parsed_packets[str(packet[IP].id)][other_node]['timestamp']}")
                print ("\n")
    print (f'***********************************************************************************\n\n\n')
    

### Graph skewed clock result


In [ ]:
parsed_packets = parsed_data['skewed']

df = pd.DataFrame(index = node_names)
pd.set_option("display.precision", 8)
for packet in parsed_packets:
    try:
        #print(packet)
        if len(parsed_packets[packet]) == len(node_names):
            df[packet] = [parsed_packets[packet]['node1']['timestamp'], parsed_packets[packet]['node2']['timestamp'], parsed_packets[packet]['node3']['timestamp'], parsed_packets[packet]['node4']['timestamp']]
    except Exception as e:
        print(e)

fig = px.line(df, x=node_names, y=df.columns, title = f'packet timestamps (skewed)', markers=True)
fig.update_layout(xaxis_title="Nodes", yaxis_title="Timestamp", legend_title="packet ID", yaxis = dict(tickformat = '.9f'))
fig.show()

### Graph corrected clock result

In [ ]:
parsed_packets = parsed_data['synced']

df = pd.DataFrame(index = node_names)
for packet in parsed_packets:
    try:
        #print(packet)
        if len(parsed_packets[packet]) == len(node_names):
            df[packet] = [parsed_packets[packet]['node1']['timestamp'], parsed_packets[packet]['node2']['timestamp'], parsed_packets[packet]['node3']['timestamp'], parsed_packets[packet]['node4']['timestamp']]
    except Exception as e:
        print(e)

fig = px.line(df, x=node_names, y=df.columns, title = f'packet timestamps (synced)', markers=True)
fig.update_layout(xaxis_title="Nodes", yaxis_title="Timestamp", legend_title="packet ID", yaxis = dict(tickformat = '.9f'))
fig.show()

### (optional) Delete the graph figures saved under `./iframe_figures`

In [ ]:
%%bash
rm iframe_figures/*